Imports

In [674]:
import csv
import pandas as pd
import numpy as np
from random import shuffle
import tensorflow as tf
import keras_nlp
import tensorflow_models as tfm
import tensorflow_text as tf_text
import tensorflow_hub as hub
from langdetect import detect
import google_transs
import pickle
import fasttext
import re
import sys
from tensorflow import keras
from keras import layers

Cleaning Text

In [650]:
text = "elene,შენ  გამახსენდი და მაგარია"

def linkRemoval(text):
    if 'http' in text:
        t = ""
        s = -1
        for i in range(0,len(text)):
            if(s!=-1 and text[i]==' '):
                s=-1  
            if(i+3 < len(text)):
                if(text[i]+text[i+1]+text[i+2]+text[i+3] == 'http'):
                    s=i
            if(s==-1):
                t+=text[i]
        text = t
    return text

def transLang(text):
    try:
        lang = detect(text)
        translator = google_transs.google_translator()
        if(lang=='en' or lang=='ru' or lang=='pl' or lang=='uk'):
            text = translator.translate(text, lang_tgt='ka')
    except:
        return text
    return text
    

convertor = {
    'a' : 'ა',
    'b' : 'ბ',
    'g' : 'გ',
    'd' : 'დ',
    'e' : 'ე',
    'v' : 'ვ',
    'z' : 'ზ',
    'i' : 'ი',
    'k' : 'კ',
    'l' : 'ლ',
    'm' : 'მ',
    'n' : 'ნ',
    'o' : 'ო',
    'p' : 'პ',
    'r' : 'რ',
    's' : 'ს',
    't' : 'ტ',
    'u' : 'უ',
    'f' : 'ფ',
    'q' : 'კ',
    'y' : 'ყ',
    'sh': 'შ',
    'ch': 'ჩ',
    'ts': 'ც',
    'c' : 'ც',
    'dz': 'ძ',
    'w' : 'ჭ',
    'x' : 'ხ',
    'j' : 'ჯ',
    'h' : 'ჰ',
}


def isLatin(text):
    latinWordCounter = 0
    geoWordCounter = 0
    for i in text:
        if((i>'a' and i<'z') or (i>'A' and i<'Z')):
            latinWordCounter += 1
        if(i>'ა' and i<'ჰ'):
            geoWordCounter += 1
    return (latinWordCounter > geoWordCounter)

def removeDuplicateLetters(text):
    i=0
    c=0
    r=""
    while(i<len(text)):
        if(i+1<len(text)):
            if(text[i]==text[i+1]):
                c+=1
            else:
                c=0
        else:
            c=0
        if(c<1):
            r+=text[i]
        i+=1
    return r

def convertionLang(text):
    text = linkRemoval(text)
    text = transLang(text)

    text = text.lower()
    text = cleanLatinText(text)
    text = cleanGeoText(text)

    text = removeDuplicateLetters(text)

    text = stemming(text)
    return text

def cleanGeoText(text):
    res = ''
    for i in text:
        if(i=='თ'):
            i='ტ'
        elif(i=='წ'):
            i='ჭ'
        elif(i=='ჟ'):
            i='ჯ'
        elif(i=='ღ'):
            i='გ'
        elif(i=='ქ'):
            i='კ'
        res += i
    return res


def removeSuffix(text):
    suffixes=['ები', 'ებო', 'ებს', 'ების', 'ებიტ', 'ტ', 'ტა', 'იანი', 'ოვანი', 'ედ', 'ტანა', 'ზე', 'ში']
    sufArr = []
    suf=""

    for i in range(0,min(5, len(text))):
        suf=text[len(text)-i-1]+suf
        sufArr.append(suf)
    
    for i in sufArr:
        if i in suffixes:
            text = text[0:len(text)-len(i)]
            break
    return text

def removePrefix(text):
    pass

def seperateText(text):
    t=""
    for i in range(0,len(text)):
        if(text[i]<'ა' or text[i]>'ჰ'):
            t+=' '
            t+=text[i]
            t+=' '
        else:
            t+=text[i]
    return t

def stemming(text):
    text = seperateText(text)
    textArr = text.split()
    textArr = [removeSuffix(x) for x in textArr]
    return ' '.join(textArr)


def cleanLatinText(text):
    res = ''
    i=0
    
    while(i<len(text)):
        c = text[i]
        if(i+1 < len(text)):
            n = text[i+1]
            if(convertor.get(c+n) != None):
                res+=convertor.get(c+n)
                i+=1
            elif(convertor.get(c) != None):
                res+=convertor.get(c)
            else:
                res+=c
        else:
            if(convertor.get(c) != None):
                res+=convertor.get(c)
            else:
                res+=c
        i+=1
    return res
newText = convertionLang(text)
print(newText)

ელენე , შენ გამახსენდი და მაგარია


In [653]:
def progressBar(n,i):
    sys.stdout.write('\r')
    j=(i+1)/n
    # the exact output you're looking for:
    sys.stdout.write("[%-20s] %d%%" % ('='*int(20*j), 100*j))
    sys.stdout.flush()
def convertData(dataName, saveName):
    data = []
    j=0
    length = 0
    with open(dataName, 'r') as f:
        reader = csv.reader(f)
        length = len(list(reader))

    with open(dataName, 'r') as f:
        reader = csv.reader(f)
        for line in enumerate(reader):
            j+=1
            if(line[0]==0): continue
            progressBar(length, j-1)
            tmp = [0, "", 0]
            tmp[0] = line[1][1]
            tmp[1] = convertionLang(line[1][2])
            if(line[1][3]=='pos'): tmp[2]=0
            if(line[1][3]=='neg'): tmp[2]=1
            if(line[1][3]=='neu'): tmp[2]=2
            data.append(tmp)
    
        print("Data Has been read and converted!")
        with open(saveName, 'wb') as ff:
            pickle.dump(data, ff)

In [654]:
convertData("train_set.csv", "train_set_mod.pckl")

[====================] 100%Data Has been read and converted!


In [655]:
convertData("test_set.csv", "test_set_mod.pckl")

[====================] 100%Data Has been read and converted!


Importing train and test sets

In [656]:
with open('train_set_mod.pckl', 'rb') as f:
    train_set = pickle.load(f)

with open('test_set_mod.pckl', 'rb') as f:
    test_set = pickle.load(f)

print(train_set[0])
print(test_set[0])

['31167', 'სიგუაც ზ უნდა დაგაყოლონ და სამივეს დედაუნდამოგიტყნა . არცერტი ხარ დასამარხი .', 0]
['20647', 'რისტვის გაიმე ბიძაშვილი და ისიც გოგო ? გმერტმა ნატელ ამყოფოს', 1]


Creating `Y` for train set and test set

In [669]:
def createY(dataset):
    y=[]
    for i in dataset:
        tmp=[0,0,0]
        n = int(i[2])
        tmp[n]=1
        y.append(tmp)
    return y

In [670]:
y_train = np.array(createY(train_set))
y_test  = np.array(createY(test_set))

Creating `X` for train set and test set

In [671]:
def createX(dataset):
    x=[]
    for i in dataset:
        x.append(i[1])
    return x

In [707]:
x_train = np.array(createX(train_set))
x_test  = np.array(createX(test_set))
print(x_train[0])
print(x_test[0])

სიგუაც ზ უნდა დაგაყოლონ და სამივეს დედაუნდამოგიტყნა . არცერტი ხარ დასამარხი .
რისტვის გაიმე ბიძაშვილი და ისიც გოგო ? გმერტმა ნატელ ამყოფოს


Word Embeddings

In [703]:
def replaceNums(s):
    # Replace numbers and symbols with language
    s = s.replace('&', ' და ')
    s = s.replace('@', ' ეთ ')
    s = s.replace('0', ' ნული ')
    s = s.replace('1', ' ერთი ')
    s = s.replace('2', ' ორი ')
    s = s.replace('3', ' სამი ')
    s = s.replace('4', ' ოთხი ')
    s = s.replace('5', ' ხუთი ')
    s = s.replace('6', ' ექვსი ')
    s = s.replace('7', ' შვიდი ')
    s = s.replace('8', ' რვა ')
    s = s.replace('9', ' ცხრა ')
    return s
replaceNums("ელენე , შენ გამახსენდი . და მაგარია😀😀")

'ელენე , შენ გამახსენდი . და მაგარია😀😀'

In [708]:
def fasttextImplementation(dataset):
    with open("comments_text.txt" , 'w') as f:
        length = len(dataset)
        j=0
        for i in dataset:
            j+=1
            progressBar(length, j)
            f.write(replaceNums(i))
            f.write('\n')

[====================] 100%

In [ ]:
fasttextImplementation(x_train)

In [709]:
skipgram = fasttext.train_unsupervised(input="comments_text.txt", model='skipgram', loss='hs')

Read 0M words
Number of words:  10125
Number of labels: 0
Progress: 100.0% words/sec/thread:   76460 lr:  0.000000 avg.loss:  7.821045 ETA:   0h 0m 0s


In [715]:
skipgram.save_model('skipgram_model.bin')

In [861]:
def xMatrix(dataset):
    x_train_mat = []
    k=0
    length = len(dataset)
    for i in dataset:
        k+=1
        tmp = []
        progressBar(length, k)
        arr = i.split()
        for j in arr:
            tmp.append(skipgram.get_word_vector(j))
        x_train_mat.append(np.array(tmp))
    x_train_mat_1 = []
    for i in x_train_mat:
        x_train_mat_1.append(np.resize(i[:32],(32,100)))
    return np.array(x_train_mat_1)

In [790]:
x_train_mat_11=xMatrix(x_train)

In [791]:
x_train_mat_11.shape

(36000, 32, 100)

In [852]:
input = keras.Input(shape=(32, 100))
decoder_input = keras.Input(shape=(32, 100))

x=tf.keras.layers.Dense(12, activation=tf.keras.layers.LeakyReLU(alpha=0.1))(input)

x_dec=tf.keras.layers.Dense(12, activation=tf.keras.layers.LeakyReLU(alpha=0.1))(decoder_input)

decoder = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=512, num_heads=12)(x_dec, x)

flat = tf.keras.layers.Flatten()(decoder)
output = layers.Dense(3, activation='softmax')(flat)

model = tf.keras.Model(inputs=[decoder_input, input], outputs=output)

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

print(model.summary())
model.fit(
    [x_train_mat_11,x_train_mat_11],
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=callbacks,
    workers=8,
    use_multiprocessing=True,
)

Model: "model_155"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_259 (InputLayer)         [(None, 32, 100)]    0           []                               
                                                                                                  
 input_258 (InputLayer)         [(None, 32, 100)]    0           []                               
                                                                                                  
 dense_271 (Dense)              (None, 32, 12)       1212        ['input_259[0][0]']              
                                                                                                  
 dense_270 (Dense)              (None, 32, 12)       1212        ['input_258[0][0]']              
                                                                                          

In [862]:
x_test_mat = xMatrix(x_test)

[====================] 100%

In [863]:
x_test_mat.shape

(9000, 32, 100)

In [955]:
from sklearn.metrics import classification_report

y_predict=model.predict([x_test_mat, x_test_mat], batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_predict, axis=1)
y_test_arg=np.argmax(y_test,axis=1)

141/141 [==============================] - 3s 20ms/step


In [956]:
print(classification_report(y_test_arg, y_pred_bool))
# np.array(y_pred_bool).shape
# y_pred_bool

              precision    recall  f1-score   support

           0       0.80      0.86      0.83      3023
           1       0.78      0.68      0.73      3018
           2       0.69      0.72      0.71      2959

    accuracy                           0.75      9000
   macro avg       0.75      0.75      0.75      9000
weighted avg       0.76      0.75      0.75      9000



In [875]:
def customTest(custom_data_origin):
    custom_data = [convertionLang(x) for x in custom_data_origin]
    custom_data = xMatrix(custom_data)
    y_pred = model.predict([custom_data, custom_data], batch_size=64, verbose=1)
    y_pred_b = np.argmax(y_pred, axis=1)
    results=[]
    for i in range(0,len(custom_data_origin)):
        results.append([custom_data_origin[i], y_pred_b[i]])
    return results

In [906]:
custom_data=['']
print(customTest(custom_data))

1/1 [==============================] - 0s 55ms/step
[['genacvalet, gilocavt axal wels', 0]]


In [957]:
data = []
j=0
length = 0
with open('processed_test_comments_formatted.csv', 'r') as f:
    reader = csv.reader(f)
    length = len(list(reader))

with open("processed_test_comments_formatted.csv", 'r') as f:
    reader = csv.reader(f)
    for line in enumerate(reader):
        j+=1
        if(line[0]==0): continue
        progressBar(length, j-1)
        tmp = [0, ""]
        tmp[0] = line[1][0]
        tmp[1] = convertionLang(line[1][4])
        data.append(tmp)

    print("\nData Has been read and converted!")
print(data[0])

[====================] 100%
Data Has been read and converted!
['5601486776576129_1450823435404030', 'გილოცავ მაგარია']


In [958]:
id_arr=[]
for i in data:
    id_arr.append(i[0])

In [959]:
bog_sent=[]
for i in data:
    bog_sent.append(i[1])

In [960]:
bog_sent = np.array(bog_sent)
x_mat_bog = xMatrix(bog_sent)

[====================] 100%

In [961]:
def bogTest(bog_data, id_array):
    
    y_pred = model.predict([bog_data, bog_data], batch_size=64, verbose=1)
    y_pred_b = np.argmax(y_pred, axis=1)
    
    results=[]
    for i in range(0,len(id_array)):
        results.append([id_array[i], y_pred_b[i]])
    
    return results

In [962]:
results = bogTest(x_mat_bog, id_arr)

17/17 [==============================] - 0s 19ms/step


In [930]:
results[0]

['5601486776576129_1450823435404030', 0]

In [931]:
with open('bog_answers.csv', 'w') as f:
    wr = csv.writer(f)
    wr.writerow(['COMMENT_ID','SENTIMENT'])
    for i in results:
        wr.writerow(i)

In [933]:
newTrainX=[]
with open('processed_train_comments_formatted.csv', 'r') as f:
    reader = csv.reader(f)
    length = len(list(reader))
    print(length)
with open("processed_train_comments_formatted.csv", 'r') as f:
    reader = csv.reader(f)
    for line in enumerate(reader):
        j+=1
        if(line[0]==0): continue
        progressBar(length, j-1)

        newTrainX.append(convertionLang(line[1][4]))

    print("\nData Has been read and converted!")

223
[=====================================================================================================================] 586%
Data Has been read and converted!


In [938]:
newTrainX

['რამიყვარს კომფორტული სესხ 😀',
 'ჩემი ბანკი ❤ ️ ❤ ️',
 'უიუიუი',
 '❤ ️ ❤ ️ ❤ ️',
 '😍',
 'ვერ ვიტან ამ ბანკს',
 'ტყუილად ნუ ლაპარაკობ 1 5 ჭელია ამ ბანკტან მაკვს ურტიერტობა და როცა დამჯიტდა სესხი მაშინ ვერ გამოვიყენე მე მაჰალიტად ამ ბანკტან ყველანარი ურტიერტობას ვჭყვე და ჩემს მეგობრებსაც ვურჩევ რო ამ ბანკტან ტავი შორს დაიკავონ',
 'ჩემი ჭკვ ❤ ️ მარა სად დაიკარგე კაცმა არ იცის 😃',
 'აფიორა და ძარცვა ხალხის ეს ყველაფერი 😒',
 'გამარტივდა კი 2 4 პროცენტ სამომხარებლო . ბანკი კი არა ჯოჯოხეტია',
 'ბრავო 🥰 🇬 🇪 🇬 🇪 🇬 🇷',
 'მაკედონია უტევდა . გოლ საკარტველოს სასარგებლოდ გავიტანე . გოლ',
 'ჭარმატებ 🏆',
 'გილოცავ',
 'გილოცავ საკარტველო 🇬 🇪 ⚽ 🥇',
 'ჭარმატებ',
 'ჭარმატებ გილოცავ 👏 ✨ ♥ ️ ♥ ️ ♥ ️',
 'აბა ფეხბურტის მაგინებლ სად არიან ეხლა . მოგებულსაც და ჭაგებულსაც ერტნაირად უნდა უგულშემატკივრო . ჭინ საკარტველო 😘',
 'როგორ მიხარია ჭარმატებ',
 'მაგარია',
 'ბრაოი',
 'გილოცავ ჩემო კვეყანა ❤ ️',
 'ზივზივაძე მვპ',
 'გილოცავ',
 '‼ ️ ❣ ️ ♥ ️ 👍 👌 👏 ♥ ️ ❣ ️ ‼ ️',
 '👏 🇬 🇪',
 '🧡',
 '🙏',
 '👏 🏼 ✊ 🏻 ✨ 😍 ❤ ️',
 '❤ ️',


In [937]:
newTrainY=[]
with open('newTrain.txt', 'r') as f:
    while True:
        score = f.readline()
        if score == '': break
        tmp=[0,0,0]
        tmp[int(score)]=1
        newTrainY.append(tmp)
print(newTrainY)

[[0, 1, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0]

In [939]:
x_newTrain_mat = xMatrix(np.array(newTrainX))
x_newTrain_mat.shape

[====================] 100%

(222, 32, 100)

In [954]:
model.fit(
    [x_newTrain_mat, x_newTrain_mat],
    np.array(newTrainY),
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=callbacks,
    workers=8,
    use_multiprocessing=True,
)

Epoch 1/50
3/3 [==============================] - 6s 311ms/step - loss: 0.3168 - accuracy: 0.7740 - val_loss: 0.4887 - val_accuracy: 0.6667
Epoch 2/50
3/3 [==============================] - 0s 69ms/step - loss: 0.2701 - accuracy: 0.7853 - val_loss: 0.5381 - val_accuracy: 0.6000
Epoch 3/50
3/3 [==============================] - 0s 74ms/step - loss: 0.2537 - accuracy: 0.7853 - val_loss: 0.5810 - val_accuracy: 0.5778
Epoch 4/50
3/3 [==============================] - 0s 67ms/step - loss: 0.2457 - accuracy: 0.7910 - val_loss: 0.5517 - val_accuracy: 0.5778
Epoch 5/50
3/3 [==============================] - 0s 71ms/step - loss: 0.2233 - accuracy: 0.8136 - val_loss: 0.4879 - val_accuracy: 0.6000
Epoch 6/50
3/3 [==============================] - 0s 68ms/step - loss: 0.2055 - accuracy: 0.8136 - val_loss: 0.4538 - val_accuracy: 0.6667
Epoch 7/50
3/3 [==============================] - 0s 71ms/step - loss: 0.2024 - accuracy: 0.8475 - val_loss: 0.4487 - val_accuracy: 0.6889
Epoch 8/50
3/3 [==========

In [950]:
with open("model_v2", 'rb') as f:
    model = pickle.load(f)
    # pickle.dump(model, f)